# Building AI Agent Bot With RAG, Langchain, and Reasoning Engine From Scratch

## Setup

* This notebook will walk you through some required setup that you need to do before starting with the materials.

* It is highly recommended to use new virtual environment when running jupyter notebook for this workshop.

## Required Software Installed Locally

* Python version 3.9, 3.10, or 3.11. **Python3.12 will not work**.

* If you are using VSCode, please install Jupyter Notebook extensions.

* Jupyter notebook. Please follow this [installation guide](https://docs.jupyter.org/en/stable/install.html). You may choose whether you want to install classic jupyter notebook or jupyterlab (the next-gen web ui for jupyter)

    * [Classic jupyter notebook installation guide](https://docs.jupyter.org/en/stable/install/notebook-classic.html)

    * [Jupyterlab installation guide](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)

* Google Cloud CLI. Please follow this [installation guide](https://cloud.google.com/sdk/docs/install-sdk)

### Installing dependencies

In [1]:
%%writefile requirements.txt

google-cloud-aiplatform
google-cloud-aiplatform[langchain]
google-cloud-aiplatform[reasoningengine]
langchain
langchain_core
langchain_community
langchain-google-vertexai==2.0.8
cloudpickle
pydantic==2.9.2
langchain-google-community
google-cloud-discoveryengine
nest-asyncio
asyncio==3.4.3
asyncpg==0.29.0
cloud-sql-python-connector[asyncpg]
langchain-google-cloud-sql-pg
numpy
pandas
pgvector
psycopg2-binary
langchain-openai
langgraph
traceloop-sdk
opentelemetry-instrumentation-google-generativeai
opentelemetry-instrumentation-langchain
opentelemetry-instrumentation-vertexai
python-dotenv

Overwriting requirements.txt


In [2]:
!pip install --upgrade -r requirements.txt

  Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached cloud_sql_python_connector-1.17.0-py3-none-any.whl.metadata (30 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of cloud-sql-python-connector[asyncpg] to determine which version is compatible with other requirements. This could take a while.
  Using cached cloud_sql_python_connector-1.16.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached cloud_sql_python_connector-1.15.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached cloud_sql_python_connector-1.14.0-py2.py3-none-any.whl.metadata (28 kB)


in case you are facing issue with installing psycopg2, please run the following command (linux only):

```
sudo apt update
sudo apt install python3-dev libpq-dev
```

You will require to restart the jupyter kernel once the dependency installed.

In [3]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Setting up Google Cloud Account

#### Recommended account setup

if you are running this in jupyter notebook locally, you may need to login to google cloud by running the following command from terminal:

```
gcloud auth login
gcloud auth application-default login
```

If you are using Google Colabs, you need to authenticate with your google account by running the following notebook cell.

> Please remember that you will need to do this on each jupyter notebook during this workshop

In [3]:
# #@markdown ###Authenticate your Google Cloud Account and enable APIs.
# # Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

## Accessing Google Cloud Credit

Please redeem your $5 USD credit that you can use for this workshop. Link for this, will be shared on the class room.

The instruction given will also require you to create a new GCP project. Create one!

## Enabling Google Service API

Before creating cloud resources (e.g. database, cloudrun services, reasoning engine, etc), first we must enable the services api.

In [4]:
# @markdown Replace the required placeholder text below. You can modify any other default values, if you like.

# please change the project id into your gcp project id you just created.
project_id = "agent-rag-451702"  # @param {type:"string"}

# you can leave this the same.
region = "us-central1"  # @param {type:"string"}

!gcloud config set project {project_id} --quiet

Updated property [core/project].


In [5]:
from googleapiclient import discovery
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=project_id)
response = request.execute()
project_number = response["projectNumber"]
project_number

'644240883738'

Here, we will enable few services:

* `aiplatform.googleapis.com` -> used for using Gemini LLM and reasoning engine
* `run.googleapis.com` -> used for deploying to cloud run
* `cloudbuild.googleapis.com` -> used for building docker image and perform the deployment

In [9]:
!gcloud config set core/disable_prompts True


!gcloud services enable artifactregistry.googleapis.com
!gcloud services enable compute.googleapis.com
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable run.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable sqladmin.googleapis.com
!gcloud services enable cloudtrace.googleapis.com

!gcloud beta services identity create --service=aiplatform.googleapis.com --project={project_id}

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:{project_number}-compute@developer.gserviceaccount.com \
    --role="roles/cloudbuild.builds.builder" -q

Updated property [core/disable_prompts].
Operation "operations/acat.p2-644240883738-de95c33c-f5d3-4729-9f18-45479ed176ca" finished successfully.
Operation "operations/acf.p2-644240883738-625d7b4b-6624-4a0b-9d0b-dbc7547d96b8" finished successfully.
Operation "operations/acat.p2-644240883738-8dea7130-c828-4201-9ad3-3cd3121eccbd" finished successfully.
Operation "operations/acf.p2-644240883738-c5625ea7-3bec-4a74-a046-30cadf66c79b" finished successfully.
Operation "operations/acf.p2-644240883738-55d886c7-5894-4453-a2ed-eb8dcec9cd5d" finished successfully.
Operation "operations/acat.p2-644240883738-eae649b3-1f78-4ad5-9448-7b9e40103a23" finished successfully.
Service identity created: service-644240883738@gcp-sa-aiplatform.iam.gserviceaccount.com
Updated IAM policy for project [agent-rag-451702].
bindings:
- members:
  - serviceAccount:644240883738-compute@developer.gserviceaccount.com
  - serviceAccount:644240883738@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- me

# Deploying Dummy API server

Later on this workshop, you will be using your AI agent to interact with api in order to get detail about an online course you provide as well as to create purchase request. Hence, we will deploy the simple stupid API to cloudrun.

If you want to see the detail, you can check the `api/` directory.

Now let's deploy the Go API to cloud run:

In [10]:
# change this registry name with an unique name
registry_name = "mkrs"  # @param {type:"string"}

!gcloud artifacts repositories create {registry_name} \
      --repository-format=docker \
      --location={region} \
      --description="devfest artifact registry" \
      --immutable-tags

registry_url = f"{region}-docker.pkg.dev/{project_id}/{registry_name}"

Create request issued for: [mkrs]
Created repository [mkrs].


We will build the docker image used by the API

In [ ]:
!gcloud builds submit api --tag {registry_url}/courses-api

Creating temporary archive of 8 file(s) totalling 15.3 KiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/Users/pabrik/.config/gcloud/logs/2025.02.09/06.16.23.532336.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [api] to [gs://gen-lang-client-0521448746_cloudbuild/source/1739056583.845295-1ffa294250044aab86681586a6fb437c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/gen-lang-client-0521448746/locations/global/builds/ffb82b4f-518c-43d9-9347-dfe786dc97a7].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/ffb82b4f-518c-43d9-9347-dfe786dc97a7?project=672065512482 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "ffb82b4f-518c-43d9-9347-dfe786dc97a7"

FETCHSOURCE
Fetching storage object: g

We will deploy the docker image to cloud run so that we can have the api up and running

In [ ]:
!gcloud run deploy courses-api --allow-unauthenticated --region {region} --quiet --image {registry_url}/courses-api

Deploying container to Cloud Run service [courses-api] in project [gen-lang-client-0521448746] region [us-central1]
Deploying...                                                                   
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      
  Deploying...                                                                 



⠛ Deploying...                                                                 



⠹ Deploying...                                                                 



⠼ Deploying...                                                                 



⠶ Deploying...                                                                 



⠧ Deploying...                                                                 



⠏ Deploying...                                                    

Once it is deployed, run the command to get the url of your dummy api. Take note because we will use it later:

In [ ]:
urls = !gcloud run services describe courses-api --region=us-central1 --format='value(status.url)'
api_url = urls[0]
print(api_url)

https://courses-api-guckng3ccq-uc.a.run.app


Testing the API

In [ ]:
!curl {api_url}/courses

[{"name":"software-security","display_name":"Software Security","description":"Learn how to secure your software","price":100,"currency":"USD"}]

# Creating Staging Bucket for AI Agent

Later, when we deploy the AI Agent, we have to provide the staging gcs bucket used to store the pickle and some other configurations of our reasoning engine. So, let's create a new empty bucket. Please change `staging_bucket_name` variable below with globally unique name.

Once the bucket created, take note the name of the bucket.

In [11]:
# change this with globaly unique name. you may add your name to make it unique. this bucket will be used later for storing the model
staging_bucket_name = "mkrs-demo-bucket" # @param {type:"string"}

!gcloud storage buckets create gs://{staging_bucket_name} --project={project_id} --location={region} --uniform-bucket-level-access

Creating gs://mkrs-demo-bucket/...


# Data Preparation

In this workshop, we are going to use written content from [OWASP CheatSheetSeries](https://github.com/OWASP/CheatSheetSeries) as the source document for our RAG. However, to reduce the cost, I already currated few files that we are going to use in `urls` variable. Instead of using all of them, we will just use few of them and build embedding with the currated files.

The source code below will just iterate over all files within `sources` directory and create a `course_content.jsonl` file containing the file contents.

In [6]:
import psycopg2
import pandas as pd

host = "lecture-dss-db.mekarsa.com"
port = 54321
database = "warehouse"
user = "lecture2024"
password = "Ojolali123"
table_name = "fact_pendapatan_daerah"


# Koneksi ke PostgreSQL
conn = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    host=host,
    port=port
)

# Jalankan query dan ubah ke DataFrame
query = """
SELECT * FROM fact_pendapatan_daerah fpd
JOIN dim_rekening dr ON fpd.kode_rekening = dr.kode_rekening
"""
df = pd.read_sql(query, conn)

conn.close()


<ipython-input-6-7606c59e6ff2>:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [20]:
df

,id_fact,kode_rekening,target_murni,totrealisasi_after,persentarget_all,kode_rekening,nama_rek,parent_kode,level_rekening
0,1437,4,4.271590e+12,2.871066e+12,67.213055,4,Pendapatan Daerah,None,1
1,1438,4.1,7.710723e+11,5.544133e+11,71.901603,4.1,Pendapatan Asli Daerah,4,2
2,1439,4.1.1,5.548600e+11,3.724990e+11,67.133876,4.1.1,Hasil Pajak Daerah,4.1,3
3,1440,4.1.1.6,5.000000e+10,4.464148e+10,89.282955,4.1.1.6,Pajak Jasa Perhotelan,4.1.1,4
4,1441,4.1.1.6.1,4.912900e+10,4.302872e+10,87.583136,4.1.1.6.1,Pajak Hotel,4.1.1.6,5
...,...,...,...,...,...,...,...,...,...
713,2150,4.3.3.1.2.4,0.000000e+00,0.000000e+00,0.000000,4.3.3.1.2.4,Pendapatan atas Pengembalian Hibah pada BUMD,4.3.3.1.2,6
714,2151,4.3.3.1.2.5,0.000000e+00,0.000000e+00,0.000000,4.3.3.1.2.5,"Pendapatan atas Pengembalian Hibah pada Badan,...",4.3.3.1.2,6
715,2152,4.3.3.1.2.6,0.000000e+00,0.000000e+00,0.000000,4.3.3.1.2.6,Pendapatan atas Pengembalian Hibah Bantuan Keu...,4.3.3.1.2,6
716,2153,4.3.3.1.3,6.000000e+10,6.578343e+10,109.639058,4.3.3.1.3,Lain lain Pendapatan Sesuai Ketentuan Peratura...,4.3.3.1,5


Let's see what is inside the `course_content.jsonl` file:

In [ ]:
import pandas as pd

df = pd.read_json('course_content.jsonl', lines=True)
df.head()

,id,title,content,file_path,slug
0,cdefd214-ffda-4c6a-85c8-e98e753baf73,Authentication Cheat Sheet,# Authentication Cheat Sheet\n\n## Introductio...,https://raw.githubusercontent.com/OWASP/CheatS...,authentication-cheat-sheet
1,084925d6-2a42-4edb-9005-a768253f264d,Authorization Cheat Sheet,# Authorization Cheat Sheet\n\n## Introduction...,https://raw.githubusercontent.com/OWASP/CheatS...,authorization-cheat-sheet
2,1349c48e-e9aa-46aa-a20d-0492d5e85363,File Upload Cheat Sheet,# File Upload Cheat Sheet\n\n## Introduction\n...,https://raw.githubusercontent.com/OWASP/CheatS...,file-upload-cheat-sheet
3,a2a2096f-1cb2-43c4-9f26-ba53c321c7ba,Forgot Password Cheat Sheet,# Forgot Password Cheat Sheet\n\n## Introducti...,https://raw.githubusercontent.com/OWASP/CheatS...,forgot-password-cheat-sheet
4,af00b636-6bb5-4c0a-8b61-a0acd6aff315,Password Storage Cheat Sheet,# Password Storage Cheat Sheet\n\n## Introduct...,https://raw.githubusercontent.com/OWASP/CheatS...,password-storage-cheat-sheet


# Creating Embedding and Vector Store

This notebook demonstrates the process of creating embeddings and setting up a vector store for a course content retrieval system.

It covers the following key steps:

1. Importing necessary libraries and creating and setting up database and its configurations
1. Connecting to either a Google Cloud SQL
1. Loading course content data from markdown files
1. Creating embeddings for the course content using a Google Gemini embedding model
1. Storing the embeddings in a vector database for efficient similarity search

Setting up few constants:

In [15]:
instance_name="mkrs-demo" # @param {type:"string"}
database_password = 'testing' # @param {type:"string"} #change this to your database password
database_name = 'testing' # @param {type:"string"} #change this to your database name
database_user = 'testing' # @param {type:"string"} #change this to your database user

# Dont update these lines below

embeddings_table_name = "course_content_embeddings"
chat_history_table_name = "chat_histories"
gemini_embedding_model = "text-embedding-004"

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"


## Setting Up PostgreSQL in Google Cloud SQL

Here will we set the default GCP project and get information about the user using the GCP account.

In [8]:
# Grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"
print(f"{current_user}")

['sugengdcahyo@gmail.com']


Before sending query to database, we will have to add required permissions for our notebook so that it can access the database:

In [15]:
print(f"Granting Cloud SQL Client role to {current_user[0]}")
# granting cloudsql client role to the current user
!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Granting Cloud SQL Client role to sugengdcahyo@gmail.com
Updated IAM policy for project [agent-rag-451702].
bindings:
- members:
  - serviceAccount:service-644240883738@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:644240883738-compute@developer.gserviceaccount.com
  - serviceAccount:644240883738@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-644240883738@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - user:sugengdcahyo@gmail.com
  role: roles/cloudsql.client
- members:
  - serviceAccount:service-644240883738@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-644240883738@containerregistry.iam.gserviceaccount.com
  role: roles/containerregistry.ServiceAgent
- members:
  - serviceAccount:644240883738-compute@developer.gserviceaccount.com
  - ser

Next, we are going to create new postgresql database from Google CloudSQL and create postgresql user/role which will be used to store the embeddings later on

In [9]:
#@markdown Create and setup a Cloud SQL PostgreSQL instance, if not done already.
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found an existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_15 \
    --region={region} --cpu=1 --memory=4GB --root-password={database_password} \
    --authorized-networks=0.0.0.0/0
# Create the database, if it does not exist.
out = !gcloud sql databases list --instance={instance_name} --filter="NAME:{database_name}" --format="value(NAME)"
if ''.join(out) == database_name:
  print("Database %s already exists, skipping creation." % database_name)
else:
  !gcloud sql databases create {database_name} --instance={instance_name}
# Create the database user for accessing the database.
!gcloud sql users create {database_user} \
  --instance={instance_name} \
  --password={database_password}

Found an existing Postgres Cloud SQL Instance!
Database testing already exists, skipping creation.
Created user [testing].


Here we are going to get the ip of postgresql we just created. Take note to the database host ip address.

In [10]:
# get the ip address of the instance
ip_addresses = !gcloud sql instances describe {instance_name} --project {project_id} --format 'value(ipAddresses.ipAddress)'
# Split the IP addresses and take the first one
database_host = ip_addresses[0].split(';')[0].strip()
print(f"Using database host: {database_host}")

Using database host: 35.239.247.104


## Prepare the embeddings

Now, we will build the embeddings from the content we have selected.

Before creating the embedding, we need to split the content of each files into chunks. This is most of the time required, especially when the content is toolong, because embedding has the limit for the number of input token it can accept.

In [35]:
!pip install --upgrade google-cloud-aiplatform


In [11]:
import pandas as pd
from vertexai.language_models import TextEmbeddingModel
from langchain.text_splitter import MarkdownTextSplitter
from langchain_core.documents import Document
from google.cloud import aiplatform

# Inisialisasi Vertex AI
location = "us-central1"

aiplatform.init(project=project_id, location=location)

# Use vertexai.language_models.TextEmbeddingModel to load the embedding model
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")

# Fungsi untuk mendapatkan embedding dari Vertex AI
def get_embedding(text):
    response = embedding_model.get_embeddings([text])
    return response[0]

# Buat Markdown Text Splitter
text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# List untuk menyimpan hasil
chunked = []

# Iterasi DataFrame
for index, row in df.iterrows():
    kode_rekening = row["kode_rekening"]
    nama_rekening = row["nama_rek"]
    content = f"Kode Rekening: {kode_rekening}\nNama Rekening: {nama_rekening}\nPendapatan: {row['target_murni']}\nRealisasi: {row['totrealisasi_after']}\nPersentase: {row['persentarget_all']}"

    # Split teks ke dalam chunk kecil
    splits = text_splitter.create_documents([content])

    for s in splits:
        metadata = {
            "kode_rekening": kode_rekening,
            "nama_rekening": nama_rekening
        }

        # Ambil embedding dari Vertex AI
        metadata["embedding"] = get_embedding(s.page_content)

        # Buat dokumen LangChain
        doc = Document(page_content=s.page_content, metadata=metadata)
        chunked.append(doc)

# Contoh output pertama
chunked[0]

Document(metadata={'kode_rekening': kode_rekening    4
kode_rekening    4
Name: 0, dtype: object, 'nama_rekening': 'Pendapatan Daerah', 'embedding': TextEmbedding(values=[0.02960214763879776, -0.015203156508505344, -0.02225613035261631, 0.010706629604101181, 0.07355363667011261, 0.02260500006377697, 0.02218550257384777, -0.06401398777961731, -0.02256166934967041, 0.0248210858553648, -0.02891496941447258, -0.009132466278970242, -0.0583990104496479, 9.912889800034463e-05, -0.002229772973805666, 0.0005700799520127475, 0.008369393646717072, -0.01962440460920334, -0.028111064806580544, -0.056292641907930374, -0.06473688036203384, 0.013326802290976048, -0.01194078754633665, 0.03180437162518501, 0.00406937999650836, -0.01592911221086979, 0.005190434865653515, -0.02846512384712696, -0.030637163668870926, 0.018438050523400307, -0.03536650165915489, 0.03554858639836311, -0.03242355212569237, -0.009869933128356934, -0.005281076766550541, -0.02853880636394024, -0.01916935294866562, 0.0012508730869

In [12]:
len(chunked)

718

Once we have the file content chunked into smaller sizes, we are going to create embedding for each chunked and store it to cloudsql.

Now let's initialize vertex ai sdk and create the embedding services.

In [13]:
from langchain_google_vertexai import VertexAIEmbeddings
import vertexai

# Initialize Vertex AI
vertexai.init(project=project_id, location=region)
# Create a Vertex AI Embeddings service
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)

In [14]:
embeddings_service

VertexAIEmbeddings(client=<vertexai.language_models.TextEmbeddingModel object at 0x7ec98047d4d0>, async_client=None, project='agent-rag-451702', location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='text-embedding-004', full_model_name=None, client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None}, api_endpoint=None, api_transport=None, default_metadata=(), additional_headers=None, client_cert_source=None, credentials=None, client_preview=None, temperature=None, max_output_tokens=None, top_p=None, top_k=None, n=1, seed=None, streaming=False, model_family=None, safety_settings=None, tuned_model_name=None, instance={'max_batch_size': 250, 'batch_size': 250, 'min_batch_size': 5, 'min_good_batch_size': 5, 'lock': <unlocked _thread.loc

Now, let's construct the embeddings and store it to the database.

On the function below we are doing these steps:
1. We are initiating a PostgresEngine. This instance of PostgresEngine will be used to handle database connection as well as authentication.
1. Then, `ainit_vectorstore_table()` will create a table which will be used to store the chucked content, its embedding, and metadata.
1. We initialize the PostgresVectorStore and provide the engine as well as the embedding service.
1. For each chunked document, we call function `aadd_documents` to create embedding and create new record on the given table.

In [18]:

from langchain_google_vertexai import VertexAIEmbeddings
import vertexai
import pandas as pd
import json


# Initialize Vertex AI
vertexai.init(project=project_id, location=region)
# Create a Vertex AI Embeddings service
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)


from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore
import uuid

async def create_vectorstore():
    engine = await PostgresEngine.afrom_instance(
        project_id,
        region,
        instance_name,
        database_name,
        user=database_user,
        password=database_password,
    )

    await engine.ainit_chat_history_table(
        table_name=chat_history_table_name
    )

    await engine.ainit_vectorstore_table(
        table_name=embeddings_table_name, vector_size=768, overwrite_existing=True
    )

    vector_store = await PostgresVectorStore.create(
        engine,
        table_name=embeddings_table_name,
        embedding_service=embeddings_service,
    )

    ids = [str(uuid.uuid4()) for i in range(len(chunked))]

    # Convert Pandas Series to JSON-serializable format before adding documents
    for doc in chunked:
        for key, value in doc.metadata.items():
            if isinstance(value, pd.Series):
                doc.metadata[key] = value.to_dict()  # Or value.tolist() if appropriate

    await vector_store.add_documents(chunked, ids=ids)

await create_vectorstore()

TypeError: Object of type TextEmbedding is not JSON serializable

In [26]:
from langchain_google_vertexai import VertexAIEmbeddings
import vertexai
import pandas as pd
import json
import numpy as np  # Import numpy


# Initialize Vertex AI
vertexai.init(project=project_id, location=region)
# Create a Vertex AI Embeddings service
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)


from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore
import uuid

async def create_vectorstore():
    engine = await PostgresEngine.afrom_instance(
        project_id,
        region,
        instance_name,
        database_name,
        user=database_user,
        password=database_password,
    )

    await engine.ainit_chat_history_table(
        table_name=chat_history_table_name
    )

    await engine.ainit_vectorstore_table(
        table_name=embeddings_table_name, vector_size=768, overwrite_existing=True
    )

    vector_store = await PostgresVectorStore.create(
        engine,
        table_name=embeddings_table_name,
        embedding_service=embeddings_service,
    )

    ids = [str(uuid.uuid4()) for i in range(len(chunked))]

    # Convert Pandas Series and TextEmbedding to JSON-serializable format before adding documents
    for doc in chunked:
        for key, value in doc.metadata.items():
            if isinstance(value, pd.Series):
                doc.metadata[key] = value.to_dict()  # Or value.tolist() if appropriate
            # Convert TextEmbedding to a list
            if isinstance(value, vertexai.language_models._language_models.TextEmbedding):
                doc.metadata[key] = value.values

    # Call add_documents without await as it's likely synchronous
    vector_store.add_documents(chunked, ids=ids)

await create_vectorstore()

Once you have the vector store, you can check the content from google cloud sql data viewer.

# Retriever

Once we have data stored in cloudsql, we need to find a way to query the data. This notebook covers how we can create and use the postgresql retriever to perform similarity search.

Similar to previous section, we will try to create PostgresEngine to connect to CloudSQL instance:

In [13]:
from langchain_google_cloud_sql_pg import PostgresEngine

instance_name="mkrs-demo"
database_name="testing"
database_password="testing"
database_user="testing"
embeddings_table_name="course_content_embeddings"

pg_engine = PostgresEngine.from_instance(
    project_id=project_id,
    instance=instance_name,
    region=region,
    database=database_name,
    user=database_password,
    password=database_password,
)

We create the vector store object by using the engine and embedding service we created earlier:

In [16]:
from langchain_google_cloud_sql_pg import PostgresVectorStore
from langchain_google_vertexai import VertexAIEmbeddings
import vertexai
import pandas as pd
import json
import numpy as np  # Import numpy


# Initialize Vertex AI
vertexai.init(project=project_id, location=region)
# Create a Vertex AI Embeddings service
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)

vector_store = PostgresVectorStore.create_sync(
            pg_engine,
            table_name=embeddings_table_name,
            embedding_service=embeddings_service,
        )
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

Let's try with some query:

In [17]:
retriever.invoke("data apa yang kamu punya?")

[Document(metadata={'kode_rekening': {'kode_rekening': '4.1.2.1.8.2'}, 'nama_rekening': 'Retribusi Penyediaan Peta Foto', 'embedding': [0.0344696082174778, -0.007454625330865383, -0.047294072806835175, -0.012055031023919582, 0.05886238440871239, 0.027880487963557243, 0.02431171014904976, -0.04355921223759651, -0.00846779067069292, 0.01586468145251274, 0.012816118076443672, 0.01357607264071703, -0.03829391300678253, -0.026567261666059494, 0.0034297965466976166, -0.019361525774002075, 0.02366507239639759, -0.015774590894579887, -0.00325488462112844, -0.04090754687786102, -0.018833057954907417, -0.013221456669270992, -0.009035448543727398, 0.024305950850248337, 3.389740413695108e-06, 0.012928166426718235, -0.01928899437189102, -0.0452263206243515, -0.014309290796518326, -0.01397411897778511, -0.028711356222629547, 0.030040206387639046, -0.05313967540860176, 0.006302352529019117, -0.005875626113265753, -0.054402947425842285, 0.011160578578710556, 0.0007350930827669799, 0.030164772644639015

In [22]:
# Example queries
query = "Berapa total pajak daerah?"
docs = retriever.invoke(query)
print(docs)


[Document(metadata={'kode_rekening': {'kode_rekening': '4.1.3.1.1.1'}, 'nama_rekening': 'Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMN', 'embedding': [0.014468416571617126, -0.019840024411678314, -0.039188362658023834, 0.031060006469488144, 0.06919091194868088, 0.03354857116937637, 0.04719628393650055, -0.055634718388319016, -0.0259989183396101, 0.028972767293453217, -0.01875510811805725, 0.011896294541656971, -0.06077931076288223, -0.020528322085738182, -0.02419540286064148, 0.006499904207885265, -0.016006743535399437, -0.023944420740008354, -0.00952987652271986, -0.04149502143263817, -0.041522782295942307, 0.0028735848609358072, -0.013567060232162476, 0.03403836861252785, 0.02453300915658474, -0.005916745401918888, -0.015292426571249962, -0.058211687952280045, -0.04308098927140236, 0.011486846953630447, -0.039171960204839706, 0.03624891862273216, -0.024603040888905525, -0.0006900458829477429, -0.013033625669777393, -0.0295989029109478, 

In [ ]:
# prompt: buatkan reasoning dari retriever sebelumnya

The provided code creates a vector database from financial data (specifically, regional income data from a PostgreSQL database) and then uses it to build a retriever. Let's break down the reasoning behind the retriever's functionality:

1. **Data Source:** The system pulls data from a PostgreSQL database table (`fact_pendapatan_daerah`) which appears to contain regional income information, including account codes, names, targets, realizations, and percentages.

2. **Data Preparation:**  The code processes the data in the following way:
    - It joins data from `fact_pendapatan_daerah` and `dim_rekening` to combine income figures with account details.
    - It then formats this data into a string representation, which includes the account code, account name, target income, actual income, and the percentage of the target achieved.
    -  Crucially, the code uses `MarkdownTextSplitter` to break down lengthy text into smaller, manageable chunks (1000 characters with a 200-character overlap). This is essential because embedding models have token limits.  Each chunk becomes a separate document.
    - For each chunk, it generates embeddings using the Vertex AI TextEmbeddingModel (`textembedding-gecko`). These embeddings transform text into numerical vectors that capture semantic meaning.
    - Metadata is associated with each chunk.  This includes the account code and name, which helps to identify the origin of each embedding when retrieving results. The embedding itself is also added as metadata.

3. **Vector Storage:** The embeddings and associated metadata are stored in a PostgreSQL database using the `PostgresVectorStore`.  This specialized vector store allows for efficient similarity searches based on the generated embeddings.

4. **Retrieval Process:**
    - The code constructs a retriever using the `PostgresVectorStore` and defines `search_kwargs={"k": 10}`.  This means that, given a query, the retriever will return the top 10 documents with embeddings most similar to the query's embedding.
    - When a query (e.g., "Berapa total pajak daerah?") is given, the retriever first generates an embedding for the query using the same Vertex AI embedding model as used for the data.
    - It then performs a similarity search in the PostgreSQL vector database, comparing the query's embedding with the embeddings of all the stored chunks.
    - The top 10 most similar chunks (based on cosine similarity, a standard metric for vector comparisons) are retrieved.  Critically, the metadata (including account codes, account names, and source text) associated with each of these top chunks are also returned.


**In summary:** The retriever's reasoning is based on semantic similarity.  By converting both the query and the stored financial data into vector representations that capture their semantic meaning, the system can quickly find the most relevant pieces of financial information for a given user query.  The metadata ensures that the context of the retrieved information (e.g., account code and name) is available. The use of chunking prevents issues arising from exceeding the embedding model's input token limit.
